# Baseten <> LightGBM example deployment

<a href="https://colab.research.google.com/github/basetenlabs/demos/blob/main/deployment/baseten_lightgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --upgrade lightgbm baseten

In [ ]:
# Model training

import lightgbm as lgb
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

def create_data():
    X, y = make_classification(n_samples=100,
                           n_informative=2,
                           n_classes=2,
                           n_features=6)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    train = lgb.Dataset(X_train, y_train)
    test = lgb.Dataset(X_test, y_test)
    return train, test

train, test = create_data()
params = {
        'boosting_type': 'gbdt',
        'objective': 'softmax',
        'metric': 'multi_logloss',
        'num_leaves': 31,
        'num_classes': 2,
        'learning_rate': 0.05,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0
}

def train_model():
    return lgb.train(params=params, train_set=train, valid_sets=test)

In [ ]:
# Model creation

my_model = train_model()

In [ ]:
# Model deployment

import baseten

api_key = "PASTE API KEY HERE"
baseten.login(api_key)

baseten.deploy(
    my_model,
    model_name="My LightGBM model"
)

In [ ]:
# After the deployment is finished, call your new model!

deployed_model_id = "PASTE VERSION ID HERE" # See deployed model page to find version ID
model_input = {"inputs": [[0, 0, 0, 0, 0, 0]]}

deployed_model = baseten.deployed_model_version_id(deployed_model_id)
deployed_model.predict(model_input)

In [ ]:
# Or call your model as an API

import requests
import json

endpoint = f"https://app.baseten.co/model_versions/{deployed_model_id}/predict"
auth = {"Authorization": f"Api-Key {api_key}"}
data = json.dumps(model_input)

resp = requests.post(
    endpoint,
    headers=auth,
    data=data
)

resp.json()